In [1]:
# spark.stop()

In [2]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 40)

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL basic example") \
    .getOrCreate()

In [ ]:
import 

### bus route

In [4]:
routeDfOriginal = spark.read.json("/home/lab01/bus_route/*.json")

In [5]:
routeDf = routeDfOriginal.select(explode(routeDfOriginal.ServiceResult.msgBody.itemList).alias("routes")).select('routes.*')

In [6]:
routeDf.show(5)

+-----+-------+----------+----------+------------+------------+--------------+-------------+------+------------------+------------------+---------+-------+---------+---+---------+-----------------------------+---------+-------+---------+
|arsId|beginTm|busRouteId|busRouteNm|   direction|fullSectDist|          gpsX|         gpsY|lastTm|              posX|              posY|routeType|sectSpd|  section|seq|  station|                    stationNm|stationNo|transYn| trnstnid|
+-----+-------+----------+----------+------------+------------+--------------+-------------+------+------------------+------------------+---------+-------+---------+---+---------+-----------------------------+---------+-------+---------+
|12469|   null| 123000010|       741|세곡동사거리|           0|126.9079256882|37.6467463641|  null|191875.29101219564| 460799.4598604017|        3|      0|        0|  1|111001112|               진관공영차고지|    12469|      N|122000699|
|35160|   null| 123000010|       741|세곡동사거리|         637|126.

In [7]:
df_route = routeDf.toPandas()

### bus

In [8]:
# - /home/lab01/bus123000010
# - /home/lab01/bus100100019
# - /home/lab01/bus100100073
# - /home/lab01/weather_after


In [9]:
bus123000010Df = spark.read.json('/home/lab01/bus123000010/*.json')
bus100100019Df = spark.read.json('/home/lab01/bus100100019/*.json')
bus100100073Df = spark.read.json('/home/lab01/bus100100073/*.json')

In [10]:
bus123000010Df.show(5)

+-------+---------+--------------+------------+----------+---------+----------+--------+-------+--------+---------+--------+--------+------------+------------------+------------------+------+--------+-------+---------+--------+---------+---------+
|busType|congetion|        dataTm|fullSectDist|      gpsX|     gpsY|isFullFlag|islastyn|isrunyn|lastStTm|lastStnId|nextStId|nextStTm|     plainNo|              posX|              posY|rtDist|sectDist|sectOrd|sectionId|stopFlag| trnstnid|    vehId|
+-------+---------+--------------+------------+----------+---------+----------+--------+-------+--------+---------+--------+--------+------------+------------------+------------------+------+--------+-------+---------+--------+---------+---------+
|      1|        0|20210810171400|       0.637|126.907926|37.646746|         0|       0|      1|       0|111001112|       0|       0|서울74사5356| 191875.3184860172|460799.41942632245|  65.8|       0|      1|111601165|       0|122000699|123060227|
|      0|  

In [11]:
 # ,'sectDist' 제외
df_bus123000010 = bus123000010Df.toPandas()[['plainNo','dataTm','fullSectDist','gpsX','gpsY','nextStId','sectionId','stopFlag','congetion']]
df_bus100100019 = bus100100019Df.toPandas()[['plainNo','dataTm','fullSectDist','gpsX','gpsY','nextStId','sectionId','stopFlag','congetion']]
df_bus100100073 = bus100100073Df.toPandas()[['plainNo','dataTm','fullSectDist','gpsX','gpsY','nextStId','sectionId','stopFlag','congetion']]

### weather

In [12]:
WeatherDf = spark.read.json('/home/lab01/weather_after')

In [13]:
WeatherDf.show(5)

+--------+--------+--------+---+---+---------+
|baseDate|baseTime|category| nx| ny|obsrValue|
+--------+--------+--------+---+---+---------+
|20210810|    1400|     PTY| 60|125|     -999|
|20210810|    1400|     REH| 60|125|     -998|
|20210810|    1400|     RN1| 60|125|   -998.9|
|20210810|    1400|     T1H| 60|125|   -998.9|
|20210810|    1400|     UUU| 60|125|    -52.9|
+--------+--------+--------+---+---+---------+
only showing top 5 rows



In [14]:
df_weather = WeatherDf.toPandas()

In [15]:
df_weather['nx'].value_counts()

61    2336
62    2336
60    2328
59    2304
Name: nx, dtype: int64

In [16]:
df_weather['ny'].value_counts()

126    2328
125    2328
127    2328
124    2320
Name: ny, dtype: int64

### 좌표변환

In [17]:
df_bus100100073

,plainNo,dataTm,fullSectDist,gpsX,gpsY,nextStId,sectionId,stopFlag,congetion
0,서울74사2506,20210811095541,0.981,126.880282,37.577746,113000026,113600767,0,0
1,서울74사1363,20210811095548,0.364,126.888812,37.579311,111000005,113604856,0,0
2,서울75사1366,20210811095545,0.794,126.895163,37.582246,111000007,111600004,0,0
3,서울74사4110,20210811095552,0.825,126.929481,37.56421,112000014,112602251,0,3
4,서울74사2432,20210811095548,2.201,126.945698,37.564674,112000050,112603553,0,3
...,...,...,...,...,...,...,...,...,...
99708,서울70사6587,20210816135655,1.37,127.020477,37.513484,101000002,121607076,0,3
99709,서울74사6205,20210816135511,3.317,127.002644,37.541475,101000002,101604644,0,3
99710,서울74사5459,20210816135649,0.07,126.960256,37.572137,112000017,112603901,1,3
99711,서울74사4605,20210816135655,1.819,126.945883,37.565236,112000017,112603446,0,3


In [18]:
# https://data-rider.blogspot.com/2015/12/c.html
from math import asin, sin, cos, tan, log
import numpy as np

NX = 149  # X축 격자점 수
NY = 253  # Y축 격자점 수

# 좌표 변환
def map_conv(lon, lat):
    class Map:
        pass
    Map.Re = 6371.00877  # 지도반경
    Map.grid = 5.0  # 격자간격 (km)
    Map.slat1 = 30.0  # 표준위도 1
    Map.slat2 = 60.0  # 표준위도 2
    Map.olon = 126.0  # 기준점 경도
    Map.olat = 38.0  # 기준점 위도
    Map.xo = 210 / Map.grid  # 기준점 X좌표
    Map.yo = 675 / Map.grid  # 기준점 Y좌표
    _x, _y = lamcproj(lon, lat, Map)
    _x = (_x + 1.5).astype(int)
    _y = (_y + 1.5).astype(int)
    return _x, _y

# Lambert Conformal Conic Projection
def lamcproj(lon, lat, Map):
    PI = np.arcsin(1.0) * 2.0
    DEGRAD = PI / 180.0

    re = Map.Re / Map.grid
    slat1 = Map.slat1 * DEGRAD
    slat2 = Map.slat2 * DEGRAD
    olon = Map.olon * DEGRAD
    olat = Map.olat * DEGRAD

    sn = np.tan(PI * 0.25 + slat2 * 0.5) / np.tan(PI * 0.25 + slat1 * 0.5)
    sn = np.log(np.cos(slat1) / np.cos(slat2)) / np.log(sn)
    sf = np.tan(PI * 0.25 + slat1 * 0.5)
    sf = pow(sf, sn) * np.cos(slat1) / sn
    ro = np.tan(PI * 0.25 + olat * 0.5)
    ro = re * sf / pow(ro, sn)

    ra = np.tan(PI * 0.25 + lat * DEGRAD * 0.5)
    ra = re * sf / pow(ra, sn)
    theta = lon * DEGRAD - olon
#     print(ra, theta)
    theta[theta > PI] -= 2.0 * PI
    theta[theta < -PI] += 2.0 * PI
    theta *= sn
    x = ((ra * np.sin(theta)) + Map.xo)
    y = ((ro - ra * np.cos(theta)) + Map.yo)
    return x, y

# df_bus100100073['nx'], df_bus100100073['ny'] = map_conv(df_bus100100073['gpsX'].astype('float'), df_bus100100073['gpsY'].astype('float'))

In [19]:
# df_bus100100073[['plainNo','dataTm','fullSectDist','gpsX','gpsY','nextStId','sectDist','sectionId','stopFlag','congetion']]

### bus, bus route 합치기

In [20]:
import pandas as pd

In [21]:
df_bus100100073[['plainNo','dataTm','fullSectDist','gpsX','gpsY','nextStId','sectionId','stopFlag','congetion']] # ,'sectDist' 제외

,plainNo,dataTm,fullSectDist,gpsX,gpsY,nextStId,sectionId,stopFlag,congetion
0,서울74사2506,20210811095541,0.981,126.880282,37.577746,113000026,113600767,0,0
1,서울74사1363,20210811095548,0.364,126.888812,37.579311,111000005,113604856,0,0
2,서울75사1366,20210811095545,0.794,126.895163,37.582246,111000007,111600004,0,0
3,서울74사4110,20210811095552,0.825,126.929481,37.56421,112000014,112602251,0,3
4,서울74사2432,20210811095548,2.201,126.945698,37.564674,112000050,112603553,0,3
...,...,...,...,...,...,...,...,...,...
99708,서울70사6587,20210816135655,1.37,127.020477,37.513484,101000002,121607076,0,3
99709,서울74사6205,20210816135511,3.317,127.002644,37.541475,101000002,101604644,0,3
99710,서울74사5459,20210816135649,0.07,126.960256,37.572137,112000017,112603901,1,3
99711,서울74사4605,20210816135655,1.819,126.945883,37.565236,112000017,112603446,0,3


In [22]:
# df_bus100100073['gpsX'] = df_bus100100073['gpsX'].astype('float')
# df_bus100100073['gpsY'].astype('float')
# df_bus100100073['nx'], df_bus100100073['ny'] = map_conv(, )

In [67]:
df_route[['busRouteId', 'busRouteNm', 'direction', 'fullSectDist', 'gpsX', 'gpsY', 'section', 'seq', 'station', 'stationNm', 'stationNo']]

,busRouteId,busRouteNm,direction,fullSectDist,gpsX,gpsY,section,seq,station,stationNm,stationNo
0,123000010,741,세곡동사거리,0,126.9079256882,37.6467463641,0,1,111001112,진관공영차고지,12469
1,123000010,741,세곡동사거리,637,126.9056673436,37.6468589258,111601165,2,218000221,동산동능모퉁이.한국지역난방공,35160
2,123000010,741,세곡동사거리,816,126.91373,37.643407,111602007,3,111000010,은평뉴타운10단지,12010
3,123000010,741,세곡동사거리,796,126.916837,37.637116,111603572,4,111000012,구파발역입구,12012
4,123000010,741,세곡동사거리,749,126.9191891249,37.6308651522,111602011,5,111000014,박석고개.신도고등학교,12014
5,123000010,741,세곡동사거리,628,126.9199961366,37.6252825924,111600013,6,111000925,세명컴퓨터고등학교,12016
6,123000010,741,세곡동사거리,692,126.9209337015,37.6193225551,111603705,7,111000927,연신내역.연서시장,12018
7,123000010,741,세곡동사거리,437,126.9239252428,37.6162184832,111600016,8,111000929,동명여고.천주교불광동성당,12020
8,123000010,741,세곡동사거리,565,126.9281552459,37.6124473901,111600018,9,111000931,불광역3.6호선,12022
9,123000010,741,세곡동사거리,438,126.9309073069,37.6092474762,111603621,10,111000933,불광역3호선.서울혁신파크,12024


In [23]:
df_route_100100073=df_route[df_route['busRouteId']=='100100073']
df_route_100100073['gpsX'] = df_route_100100073['gpsX'].astype(float)
df_route_100100073['gpsY'] = df_route_100100073['gpsY'].astype(float)
df_route_100100073['nx'], df_route_100100073['ny'] = map_conv(df_route_100100073['gpsX'], df_route_100100073['gpsY'])
df_route_100100073['seq'] = df_route_100100073['seq'].astype(int)
df_route_100100073['startSeq'] = df_route_100100073['seq'] - 1
df_route_100100073 = pd.merge(df_route_100100073[['seq','station','stationNo','stationNm','nx','ny']],df_route_100100073, how='inner', right_on='startSeq', left_on='seq')
# df_route_100100073 = df_route_100100073[['busRouteId','section','seq_x','station_x','stationNm_x','stationNo_x','nx_x','ny_x','station_y','stationNo_y','stationNm_y','nx_y','ny_y']]
df_route_100100073 = df_route_100100073[['busRouteId','section','seq_x', 'station_x','stationNm_x','nx_x','ny_x', 'station_y','stationNm_y','nx_y','ny_y']]
df_route_100100073 = df_route_100100073.rename(columns={'seq_x': 'seq'})

/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ubuntu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

In [24]:
df_route_100100073

,busRouteId,section,seq,station_x,stationNm_x,nx_x,ny_x,station_y,stationNm_y,nx_y,ny_y
0,100100073,113600767,1,113000020,서부면허시험장,58,127,113000026,월드컵파크3단지정문,58,127
1,100100073,113600709,2,113000026,월드컵파크3단지정문,58,127,113000027,월드컵파크2단지.에스플렉스센터,58,127
2,100100073,113600064,3,113000027,월드컵파크2단지.에스플렉스센터,58,127,113000192,상암DMC홍보관.YTN,58,127
3,100100073,113600711,4,113000192,상암DMC홍보관.YTN,58,127,113000204,누리꿈스퀘어.MBC,58,127
4,100100073,113604856,5,113000204,누리꿈스퀘어.MBC,58,127,113000194,월드컵파크5단지.상암중고등학교입구,58,127
5,100100073,113604943,6,113000194,월드컵파크5단지.상암중고등학교입구,58,127,113000196,상암DMC입구,58,127
6,100100073,113600770,7,113000196,상암DMC입구,58,127,113000021,DMC첨단산업센터,58,127
7,100100073,111602463,8,113000021,DMC첨단산업센터,58,127,111000003,수색교,58,127
8,100100073,111900086,9,111000003,수색교,58,127,111000005,수색역앞,58,127
9,100100073,111600004,10,111000005,수색역앞,58,127,111000007,디지털미디어시티역,58,127


In [25]:
df_bus_100100073_merge = pd.merge(df_bus100100073.drop(columns=['nextStId']),df_route_100100073, how='left', left_on='sectionId', right_on='section').drop(columns=['section'])
df_bus_100100073_merge['baseDateTime'] = df_bus_100100073_merge['dataTm'].str.slice(stop=12)
df_bus_100100073_merge = df_bus_100100073_merge.drop(columns=['dataTm'])
df_bus_100100073_merge = df_bus_100100073_merge[df_bus_100100073_merge.columns[-1:].to_list() + df_bus_100100073_merge.columns[:-1].to_list()]
df_bus_100100073_merge = df_bus_100100073_merge.sort_values(['plainNo', 'baseDateTime'])

In [26]:
df_bus_100100073_merge

,baseDateTime,plainNo,fullSectDist,gpsX,gpsY,sectionId,stopFlag,congetion,busRouteId,seq,station_x,stationNm_x,nx_x,ny_x,station_y,stationNm_y,nx_y,ny_y
44533,202108101024,서울70사6587,0.981,126.88121,37.577202,113600767,0,0,100100073,1,113000020,서부면허시험장,58,127,113000026,월드컵파크3단지정문,58,127
43819,202108101025,서울70사6587,0.981,126.885524,37.574655,113600767,0,0,100100073,1,113000020,서부면허시험장,58,127,113000026,월드컵파크3단지정문,58,127
46150,202108101026,서울70사6587,0.204,126.889228,37.573822,113600709,1,0,100100073,2,113000026,월드컵파크3단지정문,58,127,113000027,월드컵파크2단지.에스플렉스센터,58,127
46633,202108101027,서울70사6587,0.393,126.891678,37.576471,113600064,0,0,100100073,3,113000027,월드컵파크2단지.에스플렉스센터,58,127,113000192,상암DMC홍보관.YTN,58,127
52520,202108101028,서울70사6587,0.393,126.892014,37.576714,113600064,0,0,100100073,3,113000027,월드컵파크2단지.에스플렉스센터,58,127,113000192,상암DMC홍보관.YTN,58,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87232,202108181751,서울75사1653,0.542,126.888082,37.586348,111600275,0,4,100100073,87,111000004,수색교,58,127,113000022,DMC첨단산업센터,58,127
87208,202108181752,서울75사1653,0.542,126.88742,37.586927,111600275,0,4,100100073,87,111000004,수색교,58,127,113000022,DMC첨단산업센터,58,127
87136,202108181753,서울75사1653,0.667,126.884787,37.585598,113604012,1,4,100100073,88,113000022,DMC첨단산업센터,58,127,113000181,월드컵파크7단지,58,127
86558,202108181754,서울75사1653,0.667,126.884626,37.585227,113604012,1,4,100100073,88,113000022,DMC첨단산업센터,58,127,113000181,월드컵파크7단지,58,127


In [27]:
df_bus_100100073_merge.columns

Index(['baseDateTime', 'plainNo', 'fullSectDist', 'gpsX', 'gpsY', 'sectionId',
       'stopFlag', 'congetion', 'busRouteId', 'seq', 'station_x',
       'stationNm_x', 'nx_x', 'ny_x', 'station_y', 'stationNm_y', 'nx_y',
       'ny_y'],
      dtype='object')

In [28]:
# stationNo 값에 '미정차'라는 값도 들어감
df_bus_100100073_merge[df_bus_100100073_merge['plainNo']=='서울74사2506']

,baseDateTime,plainNo,fullSectDist,gpsX,gpsY,sectionId,stopFlag,congetion,busRouteId,seq,station_x,stationNm_x,nx_x,ny_x,station_y,stationNm_y,nx_y,ny_y
43338,202108100940,서울74사2506,0.825,126.928033,37.565534,112602251,0,3,100100073,16,112000008,연희104고지앞.구성산회관,59,127,112000010,서대문우체국,59,127
49382,202108100941,서울74사2506,0.825,126.929597,37.564099,112602251,0,3,100100073,16,112000008,연희104고지앞.구성산회관,59,127,112000010,서대문우체국,59,127
47022,202108100942,서울74사2506,0.396,126.93185,37.562008,112603415,0,3,100100073,17,112000010,서대문우체국,59,127,112000012,연세대앞,59,127
48722,202108100943,서울74사2506,0.243,126.935405,37.559991,112603417,0,3,100100073,18,112000012,연세대앞,59,127,112000014,세브란스병원앞,59,127
46678,202108100944,서울74사2506,0.56,126.938707,37.559982,112603422,1,3,100100073,19,112000014,세브란스병원앞,59,127,112000016,이대후문,59,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86543,202108181754,서울74사2506,0.695,127.027284,37.498292,121607992,0,3,100100073,34,121000011,지하철2호선강남역,61,125,121000009,신분당선강남역,61,125
86568,202108181755,서울74사2506,0.494,127.028401,37.496042,122600078,1,3,100100073,35,121000009,신분당선강남역,61,125,121000007,래미안아파트.파이낸셜뉴스,61,125
86618,202108181756,서울74사2506,0.494,127.028581,37.495553,122600078,1,3,100100073,35,121000009,신분당선강남역,61,125,121000007,래미안아파트.파이낸셜뉴스,61,125
87146,202108181757,서울74사2506,0.494,127.029149,37.494672,122600078,0,3,100100073,35,121000009,신분당선강남역,61,125,121000007,래미안아파트.파이낸셜뉴스,61,125


In [29]:
# nextStId는 바로 다음정류장이 아니다
# n번째 후 정류장이다
# df_bus_100100073_merge[(df_bus_100100073_merge['plainNo']=='서울74사2506') & (df_bus_100100073_merge['dataTm'].str.contains('2021081010'))][['dataTm', 'fullSectDist', 'gpsX', 'gpsY', 'nextStId', 'station_x', 'station_y', 'stationNm_x', 'stationNm_y', 'stopFlag', 'congetion', 'seq']].sort_values(['dataTm'])

In [30]:
# stopFlag가 1이 되면 stationNm_x가 현재 위치, stationNm_y가 다음 위치
# df_bus_100100073_merge[(df_bus_100100073_merge['plainNo']=='서울74사2506') & (df_bus_100100073_merge['dataTm'].str.contains('2021081218'))].sort_values(['dataTm'])

In [31]:
# df_bus_100100073_merge[(df_bus_100100073_merge['plainNo']=='서울74사2506') & ((df_bus_100100073_merge['dataTm'].str.contains('2021081211')) | (df_bus_100100073_merge['dataTm'].str.contains('2021081212')))].sort_values(['dataTm'])

### 정류장 출발, 도착 시각 찾기

In [32]:
df_bus_100100073_merge.sort_values(['plainNo', 'baseDateTime'])

,baseDateTime,plainNo,fullSectDist,gpsX,gpsY,sectionId,stopFlag,congetion,busRouteId,seq,station_x,stationNm_x,nx_x,ny_x,station_y,stationNm_y,nx_y,ny_y
44533,202108101024,서울70사6587,0.981,126.88121,37.577202,113600767,0,0,100100073,1,113000020,서부면허시험장,58,127,113000026,월드컵파크3단지정문,58,127
43819,202108101025,서울70사6587,0.981,126.885524,37.574655,113600767,0,0,100100073,1,113000020,서부면허시험장,58,127,113000026,월드컵파크3단지정문,58,127
46150,202108101026,서울70사6587,0.204,126.889228,37.573822,113600709,1,0,100100073,2,113000026,월드컵파크3단지정문,58,127,113000027,월드컵파크2단지.에스플렉스센터,58,127
46633,202108101027,서울70사6587,0.393,126.891678,37.576471,113600064,0,0,100100073,3,113000027,월드컵파크2단지.에스플렉스센터,58,127,113000192,상암DMC홍보관.YTN,58,127
52520,202108101028,서울70사6587,0.393,126.892014,37.576714,113600064,0,0,100100073,3,113000027,월드컵파크2단지.에스플렉스센터,58,127,113000192,상암DMC홍보관.YTN,58,127
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87232,202108181751,서울75사1653,0.542,126.888082,37.586348,111600275,0,4,100100073,87,111000004,수색교,58,127,113000022,DMC첨단산업센터,58,127
87208,202108181752,서울75사1653,0.542,126.88742,37.586927,111600275,0,4,100100073,87,111000004,수색교,58,127,113000022,DMC첨단산업센터,58,127
87136,202108181753,서울75사1653,0.667,126.884787,37.585598,113604012,1,4,100100073,88,113000022,DMC첨단산업센터,58,127,113000181,월드컵파크7단지,58,127
86558,202108181754,서울75사1653,0.667,126.884626,37.585227,113604012,1,4,100100073,88,113000022,DMC첨단산업센터,58,127,113000181,월드컵파크7단지,58,127


In [33]:
df_bus_100100073_merge_group = df_bus_100100073_merge.groupby(['plainNo', 'baseDateTime']).count()
df_bus_100100073_merge_group['sectionId'].value_counts()

1     94349
2      1434
3       373
4        75
5        31
6        24
7        21
9        16
10       16
8        12
11        5
15        5
12        4
13        3
14        1
Name: sectionId, dtype: int64

In [34]:
df_bus_100100073_merge_group[df_bus_100100073_merge_group['sectionId'] == 4]

fullSectDist  gpsX  gpsY  sectionId  stopFlag  \
plainNo   baseDateTime                                                  
서울70사6587 202108101213             4     4     4          4         4   
          202108131826             4     4     4          4         4   
          202108171758             4     4     4          4         4   
서울70사6588 202108121419             4     4     4          4         4   
서울70사6589 202108121531             4     4     4          4         4   
          202108131052             4     4     4          4         4   
          202108141625             4     4     4          4         4   
          202108170858             4     4     4          4         4   
          202108171740             4     4     4          4         4   
          202108181219             4     4     4          4         4   
서울74사1363 202108111706             4     4     4          4         4   
서울74사1419 202108160944             4     4     4          4         4   
          202108181532             4     4     4          4         4   
서울74사1821 202108101334             4     4     4          4         4   
          202108101613             4     4     4          4         4   
          202108121453             4     4     4          4         4   
          202108171447             4     4     4          4         4   
          202108171824             4     4     4          4         4   
서울74사2432 202108171757             4     4     4          4         4   
서울74사2506 202108120854             4     4     4          4         4   
          202108121742             4     4     4          4         4   
          202108141002             4     4     4          4         4   
          202108171636             4     4     4          4         4   
서울74사2510 202108101602             4     4     4          4         4   
          202108161033             4     4     4          4         4   
서울74사2562 202108121358             4     4     4          4         4   
서울74사2566 202108131549             4     4     4          4         4   
서울74사4110 202108101031             4     4     4          4         4   
          202108161006             4     4     4          4         4   
서울74사4605 202108161331             4     4     4          4         4   
          202108161711             4     4     4          4         4   
          202108171732             4     4     4          4         4   
          202108181348             4     4     4          4         4   
서울74사5443 202108121804             4     4     4          4         4   
서울74사5444 202108171206             4     4     4          4         4   
          202108181356             4     4     4          4         4   
서울74사5459 202108111329             4     4     4          4         4   
          202108131420             4     4     4          4         4   
          202108161257             4     4     4          4         4   
          202108171809             4     4     4          4         4   
          202108181517             4     4     4          4         4   
서울74사5757 202108180851             4     4     4          4         4   
서울74사6205 202108121009             4     4     4          4         4   
          202108131056             4     4     4          4         4   
          202108181534             4     4     4          4         4   
서울74사7208 202108121214             4     4     4          4         4   
          202108181609             4     4     4          4         4   
서울74사7215 202108101443             4     4     4          4         4   
          202108131416             4     4     4          4         4   
          202108161222             4     4     4          4         4   
          202108171604             4     4     4          4         4   
          202108171816             4     4     4          4         4   
          202108181017             4     4     4          4         4   
         

In [35]:
df_bus_100100073_merge[(df_bus_100100073_merge['plainNo']=='서울70사6587') & (df_bus_100100073_merge['baseDateTime'].str.contains('2021081012'))].sort_values('baseDateTime')

,baseDateTime,plainNo,fullSectDist,gpsX,gpsY,sectionId,stopFlag,congetion,busRouteId,seq,station_x,stationNm_x,nx_x,ny_x,station_y,stationNm_y,nx_y,ny_y
46600,202108101200,서울70사6587,0.574,127.045779,37.463275,121608545,0,3,100100073,42,121000225,하나로마트.코트라,61,124,121000249,염곡마을.서초50플러스센터,61,124
36911,202108101201,서울70사6587,0.765,127.04892,37.461686,121600461,0,3,100100073,43,121000249,염곡마을.서초50플러스센터,61,124,121000251,탑성마을,61,124
37571,202108101201,서울70사6587,0.765,127.04892,37.461686,121600461,0,3,100100073,43,121000249,염곡마을.서초50플러스센터,61,124,121000251,탑성마을,61,124
42338,202108101203,서울70사6587,0.413,127.056798,37.458184,121608716,0,3,100100073,44,121000251,탑성마을,61,124,121000987,서초포레스타입구,61,124
45363,202108101204,서울70사6587,0.447,127.063037,37.455488,121608596,1,3,100100073,46,121000986,서초포레스타2단지앞,61,124,121000254,안골마을,61,124
41813,202108101205,서울70사6587,0.447,127.063061,37.455438,121608596,1,3,100100073,46,121000986,서초포레스타2단지앞,61,124,121000254,안골마을,61,124
43452,202108101206,서울70사6587,0.447,127.063185,37.455389,121608596,1,3,100100073,46,121000986,서초포레스타2단지앞,61,124,121000254,안골마을,61,124
44418,202108101207,서울70사6587,0.447,127.063266,37.455362,121608596,1,3,100100073,46,121000986,서초포레스타2단지앞,61,124,121000254,안골마을,61,124
43137,202108101208,서울70사6587,0.447,127.063429,37.455308,121608596,1,3,100100073,46,121000986,서초포레스타2단지앞,61,124,121000254,안골마을,61,124
43661,202108101209,서울70사6587,0.447,127.063621,37.455775,121608596,1,3,100100073,46,121000986,서초포레스타2단지앞,61,124,121000254,안골마을,61,124


In [36]:
df_bus100100073[(df_bus100100073['plainNo']=='서울70사6587') & (df_bus100100073['dataTm'].str.contains('20210810121'))].sort_values('dataTm')

,plainNo,dataTm,fullSectDist,gpsX,gpsY,nextStId,sectionId,stopFlag,congetion
38716,서울70사6587,20210810121020,0.447,127.063613,37.455778,121000988,121608596,1,3
41224,서울70사6587,20210810121020,0.447,127.063613,37.455778,121000988,121608596,1,3
45573,서울70사6587,20210810121020,0.447,127.063613,37.455778,121000988,121608596,1,3
38034,서울70사6587,20210810121320,0.447,127.063614,37.455778,121000988,121608596,1,3
40036,서울70사6587,20210810121320,0.447,127.063614,37.455778,121000988,121608596,1,3
40695,서울70사6587,20210810121320,0.447,127.063614,37.455778,121000988,121608596,1,3
43683,서울70사6587,20210810121320,0.447,127.063614,37.455778,121000988,121608596,1,3
43956,서울70사6587,20210810121732,0.447,127.063614,37.455778,121000988,121608596,0,3
46749,서울70사6587,20210810121842,0.345,127.062389,37.456284,121000988,121608598,1,3
39685,서울70사6587,20210810121935,0.421,127.058987,37.457779,121000252,121600959,1,3


In [37]:
# shift 한 거 합치기
# 도착시간
# 0->1 에서 1
# 출발시간
# 1->0 에서 1
# df_bus_100100073_shift = pd.concat([df_bus_100100073_merge, df_bus_100100073_merge.shift(1)[['stopFlag', 'plainNo']].rename(columns={'stopFlag':'preStopFlag', 'plainNo':'prePlainNo'})], axis=1)
# df_bus_100100073_shift = df_bus_100100073_shift[(df_bus_100100073_shift['stopFlag']!=df_bus_100100073_shift['preStopFlag'])&(df_bus_100100073_shift['plainNo']==df_bus_100100073_shift['prePlainNo'])]
# df_bus_100100073_shift['changeFlag'] = '1'
# df_bus_100100073_change = df_bus_100100073_merge.merge(df_bus_100100073_shift[['baseDateTime', 'plainNo', 'changeFlag']], how='left', on=['baseDateTime', 'plainNo'])
# df_bus_100100073_change['changeFlag'] = df_bus_100100073_change['changeFlag'].fillna('0')
# df_bus_100100073_change_check = df_bus_100100073_change.sort_values('changeFlag').drop_duplicates(['baseDateTime', 'plainNo', 'sectionId'])
# df_bus_100100073_change_check[df_bus_100100073_change_check['changeFlag']=='0']

In [38]:
df_bus_100100073_shift = pd.concat([df_bus_100100073_merge, df_bus_100100073_merge.shift(1)[['station_x', 'plainNo']].rename(columns={'station_x':'preStation_x', 'plainNo':'prePlainNo'})], axis=1)
df_bus_100100073_shift = df_bus_100100073_shift[(df_bus_100100073_shift['station_x']!=df_bus_100100073_shift['preStation_x'])&(df_bus_100100073_shift['plainNo']==df_bus_100100073_shift['prePlainNo'])].drop(columns=['prePlainNo'])
df_bus_100100073_shift = df_bus_100100073_shift.reset_index(drop=True)
# 빼먹은 정류장 추가
df_bus_100100073_shift = pd.concat([df_bus_100100073_shift, df_bus_100100073_shift.shift(1)[['seq', 'plainNo', 'baseDateTime']].rename(columns={'seq': 'preSeq', 'plainNo': 'prePlainNo', 'baseDateTime': 'preBaseDateTime'})], axis=1)
df_bus_100100073_shift['seqDiff'] = df_bus_100100073_shift['seq']-df_bus_100100073_shift['preSeq']
# df_bus_100100073_shift = df_bus_100100073_shift[(df_bus_100100073_shift['plainNo']==df_bus_100100073_shift['prePlainNo'])&(df_bus_100100073_shift['baseDateTime'].str.slice(stop=8)==df_bus_100100073_shift['preBaseDateTime'].str.slice(stop=8))]
df_bus_100100073_shift.loc[(df_bus_100100073_shift['plainNo']!=df_bus_100100073_shift['prePlainNo'])|(df_bus_100100073_shift['baseDateTime'].str.slice(stop=8)!=df_bus_100100073_shift['preBaseDateTime'].str.slice(stop=8)), 'seqDiff'] = 0
df_bus_100100073_shift['seqDiff'] = df_bus_100100073_shift['seqDiff'].fillna(0)
df_bus_100100073_shift.drop(columns=['preStation_x', 'preSeq', 'prePlainNo', 'preBaseDateTime', 'seqDiff'])
# # df_bus_100100073_shift[(df_bus_100100073_shift['baseDateTime'].str.contains('20210814'))&(df_bus_100100073_shift['plainNo']==df_bus_100100073_shift['prePlainNo'])]['seqDiff'].value_counts()
# # df_bus_100100073_shift[df_bus_100100073_shift['plainNo']==df_bus_100100073_shift['prePlainNo']]
# # df_bus_100100073_shift[(df_bus_100100073_shift['baseDateTime'].str.contains('2021081408'))&(df_bus_100100073_shift['plainNo']==df_bus_100100073_shift['prePlainNo'])]

df_bus_100100073_shift = df_bus_100100073_shift[1:].reset_index(drop=True)
df_bus_100100073_shift['seqDiff'].value_counts()
# df_bus_100100073_shift[(df_bus_100100073_shift['plainNo']!=df_bus_100100073_shift['prePlainNo'])|(df_bus_100100073_shift['baseDateTime'].str.slice(stop=8)!=df_bus_100100073_shift['preBaseDateTime'].str.slice(stop=8))]['seqDiff']

 1.0     38641
 2.0      1691
-87.0      411
 0.0       231
-86.0        8
 3.0         8
 4.0         6
-1.0         6
 6.0         5
-85.0        3
 5.0         2
-84.0        1
-83.0        1
Name: seqDiff, dtype: int64

In [39]:

df_bus_100100073_skipped = df_bus_100100073_shift[:0].copy()

for row in range(len(df_bus_100100073_shift)):
    df_row = df_bus_100100073_shift[row:row+1].copy()
    seqDiff = int(df_row['seqDiff'])
    if seqDiff <= 1:
        continue
    for i in range(seqDiff-1):
        df_row = df_bus_100100073_shift[row:row+1].copy() # 80 seqDiff 2 78->80
        df_row['seq'] -= i + 1
        df_row['seqDiff'] = 1
        df_row = df_row.drop(columns=['busRouteId', 'sectionId', 'station_x', 'stationNm_x', 'nx_x', 'ny_x', 'station_y', 'stationNm_y', 'nx_y', 'ny_y']).merge(df_route_100100073).rename(columns={'section': 'sectionId'})
        df_bus_100100073_skipped = pd.concat([df_bus_100100073_skipped, df_row], axis=0)
df_bus_100100073_complete = pd.concat([df_bus_100100073_shift, df_bus_100100073_skipped], axis=0).sort_values(['plainNo', 'baseDateTime', 'seq']).reset_index(drop=True)
df_bus_100100073_complete = pd.concat([df_bus_100100073_complete, df_bus_100100073_complete.shift(-1)[['baseDateTime', 'plainNo']].rename(columns={'baseDateTime': 'nextBaseDateTime', 'plainNo': 'nextPlainNo'})], axis=1)


df_bus_100100073_complete['baseDateHour'] = df_bus_100100073_complete['baseDateTime'].str.slice(stop=10)
df_bus_100100073_complete = df_bus_100100073_complete.rename(columns={'baseDateTime': 'baseDateMinute'})
df_bus_100100073_complete['interval'] = (pd.to_datetime(df_bus_100100073_complete['nextBaseDateTime'], format='%Y%m%d%H%M', errors='coerce') - pd.to_datetime(df_bus_100100073_complete['baseDateMinute'], format='%Y%m%d%H%M', errors='coerce'))
df_bus_100100073_complete = df_bus_100100073_complete.dropna(subset=['interval'])
df_bus_100100073_complete['interval'] = df_bus_100100073_complete['interval'].astype(int) // 60000000000
df_bus_100100073_complete = df_bus_100100073_complete[(df_bus_100100073_complete['seqDiff']>0)&(df_bus_100100073_complete['plainNo']==df_bus_100100073_complete['nextPlainNo'])&(df_bus_100100073_complete['baseDateMinute'].str.slice(stop=8)==df_bus_100100073_complete['nextBaseDateTime'].str.slice(stop=8))]


In [40]:
# df_bus_100100073_complete_ = df_bus_100100073_complete
# df_bus_100100073_complete = df_bus_100100073_complete_

In [41]:
df_bus_100100073_complete.head(20)

,baseDateMinute,plainNo,fullSectDist,gpsX,gpsY,sectionId,stopFlag,congetion,busRouteId,seq,station_x,stationNm_x,nx_x,ny_x,station_y,stationNm_y,nx_y,ny_y,preStation_x,preSeq,prePlainNo,preBaseDateTime,seqDiff,nextBaseDateTime,nextPlainNo,baseDateHour,interval
0,202108101027,서울70사6587,0.393,126.891678,37.576471,113600064,0,0,100100073,3,113000027,월드컵파크2단지.에스플렉스센터,58,127,113000192,상암DMC홍보관.YTN,58,127,113000026,2.0,서울70사6587,202108101026,1.0,202108101029,서울70사6587,2021081010,2
1,202108101029,서울70사6587,0.187,126.891614,37.577691,113600711,1,0,100100073,4,113000192,상암DMC홍보관.YTN,58,127,113000204,누리꿈스퀘어.MBC,58,127,113000027,3.0,서울70사6587,202108101027,1.0,202108101030,서울70사6587,2021081010,1
2,202108101030,서울70사6587,0.364,126.889286,37.579041,113604856,0,0,100100073,5,113000204,누리꿈스퀘어.MBC,58,127,113000194,월드컵파크5단지.상암중고등학교입구,58,127,113000192,4.0,서울70사6587,202108101029,1.0,202108101032,서울70사6587,2021081010,2
3,202108101032,서울70사6587,0.493,126.885259,37.581347,113604943,0,0,100100073,6,113000194,월드컵파크5단지.상암중고등학교입구,58,127,113000196,상암DMC입구,58,127,113000204,5.0,서울70사6587,202108101030,1.0,202108101034,서울70사6587,2021081010,2
4,202108101034,서울70사6587,0.331,126.884084,37.5847,113600770,0,0,100100073,7,113000196,상암DMC입구,58,127,113000021,DMC첨단산업센터,58,127,113000194,6.0,서울70사6587,202108101032,1.0,202108101035,서울70사6587,2021081010,1
5,202108101035,서울70사6587,0.448,126.885558,37.585771,111602463,0,0,100100073,8,113000021,DMC첨단산업센터,58,127,111000003,수색교,58,127,113000196,7.0,서울70사6587,202108101034,1.0,202108101037,서울70사6587,2021081010,2
6,202108101037,서울70사6587,0.512,126.88848,37.585785,111900086,1,0,100100073,9,111000003,수색교,58,127,111000005,수색역앞,58,127,113000021,8.0,서울70사6587,202108101035,1.0,202108101039,서울70사6587,2021081010,2
7,202108101039,서울70사6587,0.794,126.895414,37.58218,111600004,0,0,100100073,10,111000005,수색역앞,58,127,111000007,디지털미디어시티역,58,127,111000003,9.0,서울70사6587,202108101037,1.0,202108101041,서울70사6587,2021081010,2
8,202108101041,서울70사6587,0.533,126.900549,37.578916,112602620,0,0,100100073,11,111000007,디지털미디어시티역,58,127,112000001,북가좌동삼거리,59,127,111000005,10.0,서울70사6587,202108101039,1.0,202108101042,서울70사6587,2021081010,1
9,202108101042,서울70사6587,0.619,126.904464,37.575563,112602628,1,3,100100073,12,112000001,북가좌동삼거리,59,127,112000003,DMC래미안e.편한세상요진아파트,59,127,111000007,11.0,서울70사6587,202108101041,1.0,202108101044,서울70사6587,2021081010,2


In [42]:
pd.to_datetime('202108140849', format='%Y%m%d%H%M', errors='coerce') - pd.to_datetime('202108140849', format='%Y%m%d%H%M', errors='coerce')

Timedelta('0 days 00:00:00')

In [43]:
df_bus_100100073_complete

,baseDateMinute,plainNo,fullSectDist,gpsX,gpsY,sectionId,stopFlag,congetion,busRouteId,seq,station_x,stationNm_x,nx_x,ny_x,station_y,stationNm_y,nx_y,ny_y,preStation_x,preSeq,prePlainNo,preBaseDateTime,seqDiff,nextBaseDateTime,nextPlainNo,baseDateHour,interval
0,202108101027,서울70사6587,0.393,126.891678,37.576471,113600064,0,0,100100073,3,113000027,월드컵파크2단지.에스플렉스센터,58,127,113000192,상암DMC홍보관.YTN,58,127,113000026,2.0,서울70사6587,202108101026,1.0,202108101029,서울70사6587,2021081010,2
1,202108101029,서울70사6587,0.187,126.891614,37.577691,113600711,1,0,100100073,4,113000192,상암DMC홍보관.YTN,58,127,113000204,누리꿈스퀘어.MBC,58,127,113000027,3.0,서울70사6587,202108101027,1.0,202108101030,서울70사6587,2021081010,1
2,202108101030,서울70사6587,0.364,126.889286,37.579041,113604856,0,0,100100073,5,113000204,누리꿈스퀘어.MBC,58,127,113000194,월드컵파크5단지.상암중고등학교입구,58,127,113000192,4.0,서울70사6587,202108101029,1.0,202108101032,서울70사6587,2021081010,2
3,202108101032,서울70사6587,0.493,126.885259,37.581347,113604943,0,0,100100073,6,113000194,월드컵파크5단지.상암중고등학교입구,58,127,113000196,상암DMC입구,58,127,113000204,5.0,서울70사6587,202108101030,1.0,202108101034,서울70사6587,2021081010,2
4,202108101034,서울70사6587,0.331,126.884084,37.5847,113600770,0,0,100100073,7,113000196,상암DMC입구,58,127,113000021,DMC첨단산업센터,58,127,113000194,6.0,서울70사6587,202108101032,1.0,202108101035,서울70사6587,2021081010,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42766,202108181743,서울75사1653,0.611,126.91029,37.571552,112602626,0,4,100100073,83,112000004,DMC래미안e.편한세상요진아파트,59,127,112000002,북가좌동삼거리,59,127,112000006,82.0,서울75사1653,202108181742,1.0,202108181745,서울75사1653,2021081817,2
42767,202108181745,서울75사1653,0.497,126.905075,37.575279,111603533,0,4,100100073,84,112000002,북가좌동삼거리,59,127,111000008,디지털미디어시티역,58,127,112000004,83.0,서울75사1653,202108181743,1.0,202108181746,서울75사1653,2021081817,1
42768,202108181746,서울75사1653,0.821,126.901214,37.578624,111600008,1,4,100100073,85,111000008,디지털미디어시티역,58,127,111000006,수색역앞,58,127,112000002,84.0,서울75사1653,202108181745,1.0,202108181749,서울75사1653,2021081817,3
42769,202108181749,서울75사1653,0.507,126.894103,37.583047,111900090,1,4,100100073,86,111000006,수색역앞,58,127,111000004,수색교,58,127,111000008,85.0,서울75사1653,202108181746,1.0,202108181751,서울75사1653,2021081817,2


In [44]:
df_bus_100100073_complete['interval'].value_counts()

2      13488
1      12081
3       8099
4       2946
0       1758
5       1332
6        654
7        304
9        187
8        168
10       139
11        78
12        74
14        43
13        42
59        24
54        22
51        21
55        20
61        20
56        20
57        20
71        19
16        19
15        19
58        18
60        15
52        14
62        14
84        10
64        10
67        10
17        10
48         9
66         9
19         8
49         8
45         8
53         8
63         8
18         7
46         7
70         7
47         7
68         7
50         7
69         6
73         6
44         5
65         5
72         5
41         5
76         4
43         4
42         4
88         3
87         3
79         3
21         2
91         2
92         2
77         2
78         2
74         2
80         2
39         1
90         1
94         1
96         1
75         1
35         1
99         1
36         1
20         1
81         1
37         1
24         1

In [45]:
df_bus_100100073_complete[df_bus_100100073_complete['interval']>10]

,baseDateMinute,plainNo,fullSectDist,gpsX,gpsY,sectionId,stopFlag,congetion,busRouteId,seq,station_x,stationNm_x,nx_x,ny_x,station_y,stationNm_y,nx_y,ny_y,preStation_x,preSeq,prePlainNo,preBaseDateTime,seqDiff,nextBaseDateTime,nextPlainNo,baseDateHour,interval
43,202108101204,서울70사6587,0.447,127.063037,37.455488,121608596,1,3,100100073,46,121000986,서초포레스타2단지앞,61,124,121000254,안골마을,61,124,121000251,44.0,서울70사6587,202108101203,2.0,202108101218,서울70사6587,2021081012,14
85,202108101353,서울70사6587,0.667,126.883183,37.584183,113604012,0,3,100100073,88,113000022,DMC첨단산업센터,58,127,113000181,월드컵파크7단지,58,127,111000004,87.0,서울70사6587,202108101351,1.0,202108101436,서울70사6587,2021081013,43
223,202108111028,서울70사6587,0.667,126.884245,37.584902,113604012,0,3,100100073,88,113000022,DMC첨단산업센터,58,127,113000181,월드컵파크7단지,58,127,111000004,87.0,서울70사6587,202108111026,1.0,202108111126,서울70사6587,2021081110,58
311,202108111455,서울70사6587,0.667,126.882313,37.583549,113604012,0,3,100100073,88,113000022,DMC첨단산업센터,58,127,113000181,월드컵파크7단지,58,127,111000004,87.0,서울70사6587,202108111452,1.0,202108111600,서울70사6587,2021081114,65
433,202108121125,서울70사6587,0.667,126.88165,37.583135,113604012,0,3,100100073,88,113000022,DMC첨단산업센터,58,127,113000181,월드컵파크7단지,58,127,111000004,87.0,서울70사6587,202108121121,1.0,202108121219,서울70사6587,2021081211,54
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42549,202108171650,서울75사1653,0.667,126.885317,37.585752,113604012,1,4,100100073,88,113000022,DMC첨단산업센터,58,127,113000181,월드컵파크7단지,58,127,111000004,87.0,서울75사1653,202108171647,1.0,202108171754,서울75사1653,2021081716,64
42568,202108171829,서울75사1653,0.56,126.940033,37.560086,112603422,0,3,100100073,19,112000014,세브란스병원앞,59,127,112000016,이대후문,59,127,112000012,18.0,서울75사1653,202108171826,1.0,202108171841,서울75사1653,2021081718,12
42595,202108180916,서울75사1653,0.667,126.885068,37.585544,113604012,1,3,100100073,88,113000022,DMC첨단산업센터,58,127,113000181,월드컵파크7단지,58,127,111000004,87.0,서울75사1653,202108180913,1.0,202108181006,서울75사1653,2021081809,50
42626,202108181103,서울75사1653,2.673,127.006218,37.535319,121600088,1,3,100100073,31,102000071,순천향대학병원.한남오거리,60,126,121000015,신사역.푸른저축은행,61,126,101000328,30.0,서울75사1653,202108181059,1.0,202108181117,서울75사1653,2021081811,14


In [46]:
# sorted(list(df_bus_100100073_shift['gpsX'].unique()))

In [68]:
df_bus_100100073_shift[(df_bus_100100073_shift['baseDateTime'].str.contains('2021081408'))&(df_bus_100100073_shift['plainNo']==df_bus_100100073_shift['prePlainNo'])][['baseDateTime', 'plainNo', 'sectionId', 'stopFlag', 'seq', 'stationNm_x', 'stationNm_y', 'preStation_x', 'preSeq', 'prePlainNo', 'preBaseDateTime', 'seqDiff']]

,baseDateTime,plainNo,sectionId,stopFlag,seq,stationNm_x,stationNm_y,preStation_x,preSeq,prePlainNo,preBaseDateTime,seqDiff
2192,202108140843,서울70사6588,100601959,1,69,광화문,광화문.금호아시아나본관앞,113000022,88.0,서울70사6588,202108131742,0.0
2193,202108140845,서울70사6588,101600128,1,71,서울역사박물관.경교장.강북삼성병원,서대문역사거리.적십자병원,100000385,69.0,서울70사6588,202108140843,2.0
2194,202108140846,서울70사6588,112603437,1,72,서대문역사거리.적십자병원,서대문.서울시교육청,100000028,71.0,서울70사6588,202108140845,1.0
2195,202108140848,서울70사6588,112603900,0,73,서대문.서울시교육청,독립문(가상),100000368,72.0,서울70사6588,202108140846,1.0
2196,202108140849,서울70사6588,112603446,1,75,독립문공원.극동아파트,이대후문,100000073,73.0,서울70사6588,202108140848,2.0
2197,202108140852,서울70사6588,112602882,1,76,이대후문,세브란스병원앞,112000187,75.0,서울70사6588,202108140849,1.0
2198,202108140854,서울70사6588,112602268,0,77,세브란스병원앞,연세대앞,112000017,76.0,서울70사6588,202108140852,1.0
2199,202108140855,서울70사6588,112603455,1,78,연세대앞,서대문우체국,112000015,77.0,서울70사6588,202108140854,1.0
2200,202108140857,서울70사6588,112603457,1,79,서대문우체국,연희104고지앞.구성산회관,112000013,78.0,서울70사6588,202108140855,1.0
2201,202108140858,서울70사6588,112603851,1,80,연희104고지앞.구성산회관,사천교,112000011,79.0,서울70사6588,202108140857,1.0


In [70]:
df_bus_100100073_complete[(df_bus_100100073_complete['baseDateMinute'].str.contains('2021081408'))&(df_bus_100100073_complete['plainNo']==df_bus_100100073_complete['prePlainNo'])][['baseDateMinute', 'plainNo', 'sectionId', 'stopFlag', 'seq', 'stationNm_x', 'stationNm_y', 'nextPlainNo', 'nextBaseDateTime', 'interval']]

,baseDateMinute,plainNo,sectionId,stopFlag,seq,stationNm_x,stationNm_y,nextPlainNo,nextBaseDateTime,interval
2299,202108140845,서울70사6588,100601982,1,70,광화문.금호아시아나본관앞,서울역사박물관.경교장.강북삼성병원,서울70사6588,202108140845,0
2300,202108140845,서울70사6588,101600128,1,71,서울역사박물관.경교장.강북삼성병원,서대문역사거리.적십자병원,서울70사6588,202108140846,1
2301,202108140846,서울70사6588,112603437,1,72,서대문역사거리.적십자병원,서대문.서울시교육청,서울70사6588,202108140848,2
2302,202108140848,서울70사6588,112603900,0,73,서대문.서울시교육청,독립문(가상),서울70사6588,202108140849,1
2303,202108140849,서울70사6588,112603901,1,74,독립문(가상),독립문공원.극동아파트,서울70사6588,202108140849,0
2304,202108140849,서울70사6588,112603446,1,75,독립문공원.극동아파트,이대후문,서울70사6588,202108140852,3
2305,202108140852,서울70사6588,112602882,1,76,이대후문,세브란스병원앞,서울70사6588,202108140854,2
2306,202108140854,서울70사6588,112602268,0,77,세브란스병원앞,연세대앞,서울70사6588,202108140855,1
2307,202108140855,서울70사6588,112603455,1,78,연세대앞,서대문우체국,서울70사6588,202108140857,2
2308,202108140857,서울70사6588,112603457,1,79,서대문우체국,연희104고지앞.구성산회관,서울70사6588,202108140858,1


In [49]:
df_bus_100100073_skipped[(df_bus_100100073_skipped['baseDateTime'].str.contains('2021081408'))&(df_bus_100100073_skipped['plainNo']==df_bus_100100073_skipped['prePlainNo'])]

,baseDateTime,plainNo,fullSectDist,gpsX,gpsY,sectionId,stopFlag,congetion,busRouteId,seq,station_x,stationNm_x,nx_x,ny_x,station_y,stationNm_y,nx_y,ny_y,preStation_x,preSeq,prePlainNo,preBaseDateTime,seqDiff
0,202108140845,서울70사6588,0.457,126.971521,37.570049,100601982,1,3,100100073,70,100000029,광화문.금호아시아나본관앞,60,127,100000028,서울역사박물관.경교장.강북삼성병원,60,127,100000385,69.0,서울70사6588,202108140843,1.0
0,202108140849,서울70사6588,1.819,126.959319,37.571853,112603901,1,3,100100073,74,112000438,독립문(가상),59,127,112000187,독립문공원.극동아파트,59,127,100000073,73.0,서울70사6588,202108140848,1.0
0,202108140851,서울74사1363,0.464,126.97038,37.569135,101604199,1,3,100100073,23,101000262,서대문역사거리.농협중앙회,60,127,100000380,서울역사박물관.경희궁앞,60,127,112000051,22.0,서울74사1363,202108140848,1.0
0,202108140853,서울74사7215,0.364,126.889637,37.578988,113600711,1,0,100100073,4,113000192,상암DMC홍보관.YTN,58,127,113000204,누리꿈스퀘어.MBC,58,127,113000027,3.0,서울74사7215,202108140850,1.0
0,202108140858,서울74사7215,0.448,126.88498,37.585351,113600770,1,0,100100073,7,113000196,상암DMC입구,58,127,113000021,DMC첨단산업센터,58,127,113000194,6.0,서울74사7215,202108140855,1.0
0,202108140858,서울74사7608,0.393,126.890647,37.575267,113600709,1,0,100100073,2,113000026,월드컵파크3단지정문,58,127,113000027,월드컵파크2단지.에스플렉스센터,58,127,113000020,1.0,서울74사7608,202108140855,1.0
0,202108140857,서울75사1376,0.513,127.049156,37.462002,121600960,1,3,100100073,49,121000252,탑성마을,61,124,121000250,염곡마을.서초50플러스센터,61,124,121000988,48.0,서울75사1376,202108140856,1.0
0,202108140857,서울75사1653,1.819,126.959319,37.571853,112603901,1,3,100100073,74,112000438,독립문(가상),59,127,112000187,독립문공원.극동아파트,59,127,100000073,73.0,서울75사1653,202108140855,1.0


In [50]:
df_bus_100100073_skipped

,baseDateTime,plainNo,fullSectDist,gpsX,gpsY,sectionId,stopFlag,congetion,busRouteId,seq,station_x,stationNm_x,nx_x,ny_x,station_y,stationNm_y,nx_y,ny_y,preStation_x,preSeq,prePlainNo,preBaseDateTime,seqDiff
0,202108101204,서울70사6587,0.447,127.063037,37.455488,121600956,1,3,100100073,45,121000987,서초포레스타입구,61,124,121000986,서초포레스타2단지앞,61,124,121000251,44.0,서울70사6587,202108101203,1.0
0,202108101327,서울70사6587,1.819,126.959552,37.571866,112603901,1,3,100100073,74,112000438,독립문(가상),59,127,112000187,독립문공원.극동아파트,59,127,100000073,73.0,서울70사6587,202108101324,1.0
0,202108101720,서울70사6587,0.429,126.986247,37.570257,100602423,1,4,100100073,66,100000001,종로2가사거리,60,127,100000389,종로2가,60,127,101000002,65.0,서울70사6587,202108101717,1.0
0,202108101727,서울70사6587,0.457,126.971794,37.570149,100601982,1,4,100100073,70,100000029,광화문.금호아시아나본관앞,60,127,100000028,서울역사박물관.경교장.강북삼성병원,60,127,100000385,69.0,서울70사6587,202108101725,1.0
0,202108101735,서울70사6587,1.819,126.959568,37.571877,112603901,1,4,100100073,74,112000438,독립문(가상),59,127,112000187,독립문공원.극동아파트,59,127,100000073,73.0,서울70사6587,202108101733,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,202108181253,서울75사1653,0.429,126.986397,37.570271,100602423,1,3,100100073,66,100000001,종로2가사거리,60,127,100000389,종로2가,60,127,101000002,65.0,서울75사1653,202108181250,1.0
0,202108181306,서울75사1653,1.819,126.959444,37.571795,112603901,1,3,100100073,74,112000438,독립문(가상),59,127,112000187,독립문공원.극동아파트,59,127,100000073,73.0,서울75사1653,202108181301,1.0
0,202108181433,서울75사1653,0.448,126.884989,37.585357,113600770,1,0,100100073,7,113000196,상암DMC입구,58,127,113000021,DMC첨단산업센터,58,127,113000194,6.0,서울75사1653,202108181432,1.0
0,202108181609,서울75사1653,0.447,127.063037,37.455488,121600956,1,3,100100073,45,121000987,서초포레스타입구,61,124,121000986,서초포레스타2단지앞,61,124,121000251,44.0,서울75사1653,202108181608,1.0


In [51]:
df_bus_100100073_complete.to_csv('./df_bus_100100073_complete.json')

In [52]:
# weather 출발점 기준으로 merge

### weather 변환

In [53]:
df_weather

,baseDate,baseTime,category,nx,ny,obsrValue
0,20210810,1400,PTY,60,125,-999
1,20210810,1400,REH,60,125,-998
2,20210810,1400,RN1,60,125,-998.9
3,20210810,1400,T1H,60,125,-998.9
4,20210810,1400,UUU,60,125,-52.9
...,...,...,...,...,...,...
9299,20210816,1500,T1H,59,126,30.9
9300,20210816,1500,UUU,59,126,-1
9301,20210816,1500,VEC,59,126,85
9302,20210816,1500,VVV,59,126,0


In [54]:
df_weather_ = df_weather
df_weather_['baseDateTime'] = df_weather_['baseDate'] + df_weather_['baseTime'].str.slice(stop=2)
df_weather_ = df_weather_.drop(columns=['baseDate', 'baseTime'])

df_weather_time_loc = df_weather_[['baseDateTime', 'nx', 'ny']].drop_duplicates()
for category in df_weather_['category'].unique():
    df_weather_time_loc = df_weather_time_loc.merge(df_weather_[df_weather_['category']==category], on=['baseDateTime', 'nx', 'ny']).drop(columns='category')
    df_weather_time_loc = df_weather_time_loc.rename(columns={'obsrValue': category})
# 이후 다른 REH, RN!, ... 에 대해 똑같이 만든 후 baseDateTime	nx	ny 에 대하여 merge
df_weather_time_loc = df_weather_time_loc.rename(columns={'baseDateTime': 'baseDateHour'})
df_weather_time_loc['nx'] = df_weather_time_loc['nx'].astype(int)
df_weather_time_loc['ny'] = df_weather_time_loc['ny'].astype(int)

In [55]:
df_weather_['category'].unique()

array(['PTY', 'REH', 'RN1', 'T1H', 'UUU', 'VEC', 'VVV', 'WSD'],
      dtype=object)

In [56]:
df_weather_time_loc['REH'].value_counts()

55      59
51      58
53      55
57      48
49      48
47      47
58      47
61      45
54      45
59      42
52      39
62      37
56      36
60      35
50      34
45      31
48      29
64      28
63      27
44      25
46      22
69      22
68      20
65      18
66      17
67      17
-998    16
70      15
42      12
71      12
76      11
43      10
39       9
73       9
92       8
41       8
72       8
82       8
35       7
40       7
75       6
79       6
93       6
74       5
84       5
77       5
78       5
100      5
83       4
81       4
36       4
85       4
37       4
94       3
80       3
38       3
88       3
91       3
87       3
34       2
86       2
96       2
99       2
90       1
32       1
89       1
Name: REH, dtype: int64

### bus전체와 weather 병합

In [71]:
df_weather_time_loc.sort_values('baseDateHour')

,baseDateHour,nx,ny,PTY,REH,RN1,T1H,UUU,VEC,VVV,WSD
26,2021081008,60,124,0,65,0,27.4,-0.5,159,1.6,1.7
525,2021081008,61,126,0,58,0,28,-0.8,144,1.3,1.6
526,2021081008,61,124,0,67,0,27.5,-1.5,125,1.1,2
527,2021081008,59,127,0,61,0,27.5,-1,151,1.9,2.2
32,2021081008,61,127,0,51,0,28.9,-0.2,170,1.8,1.8
...,...,...,...,...,...,...,...,...,...,...,...
923,2021081813,59,124,0,51,0,29.4,-1.1,31,-1.9,2.3
922,2021081813,59,126,0,59,0,28.2,-2.8,63,-1.4,3.3
931,2021081813,62,127,0,49,0,29.2,-3.1,115,1.5,3.5
925,2021081813,59,127,0,62,0,27.5,-1.3,102,0.3,1.4


In [58]:
df_bus_100100073_complete

,baseDateMinute,plainNo,fullSectDist,gpsX,gpsY,sectionId,stopFlag,congetion,busRouteId,seq,station_x,stationNm_x,nx_x,ny_x,station_y,stationNm_y,nx_y,ny_y,preStation_x,preSeq,prePlainNo,preBaseDateTime,seqDiff,nextBaseDateTime,nextPlainNo,baseDateHour,interval
0,202108101027,서울70사6587,0.393,126.891678,37.576471,113600064,0,0,100100073,3,113000027,월드컵파크2단지.에스플렉스센터,58,127,113000192,상암DMC홍보관.YTN,58,127,113000026,2.0,서울70사6587,202108101026,1.0,202108101029,서울70사6587,2021081010,2
1,202108101029,서울70사6587,0.187,126.891614,37.577691,113600711,1,0,100100073,4,113000192,상암DMC홍보관.YTN,58,127,113000204,누리꿈스퀘어.MBC,58,127,113000027,3.0,서울70사6587,202108101027,1.0,202108101030,서울70사6587,2021081010,1
2,202108101030,서울70사6587,0.364,126.889286,37.579041,113604856,0,0,100100073,5,113000204,누리꿈스퀘어.MBC,58,127,113000194,월드컵파크5단지.상암중고등학교입구,58,127,113000192,4.0,서울70사6587,202108101029,1.0,202108101032,서울70사6587,2021081010,2
3,202108101032,서울70사6587,0.493,126.885259,37.581347,113604943,0,0,100100073,6,113000194,월드컵파크5단지.상암중고등학교입구,58,127,113000196,상암DMC입구,58,127,113000204,5.0,서울70사6587,202108101030,1.0,202108101034,서울70사6587,2021081010,2
4,202108101034,서울70사6587,0.331,126.884084,37.5847,113600770,0,0,100100073,7,113000196,상암DMC입구,58,127,113000021,DMC첨단산업센터,58,127,113000194,6.0,서울70사6587,202108101032,1.0,202108101035,서울70사6587,2021081010,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42766,202108181743,서울75사1653,0.611,126.91029,37.571552,112602626,0,4,100100073,83,112000004,DMC래미안e.편한세상요진아파트,59,127,112000002,북가좌동삼거리,59,127,112000006,82.0,서울75사1653,202108181742,1.0,202108181745,서울75사1653,2021081817,2
42767,202108181745,서울75사1653,0.497,126.905075,37.575279,111603533,0,4,100100073,84,112000002,북가좌동삼거리,59,127,111000008,디지털미디어시티역,58,127,112000004,83.0,서울75사1653,202108181743,1.0,202108181746,서울75사1653,2021081817,1
42768,202108181746,서울75사1653,0.821,126.901214,37.578624,111600008,1,4,100100073,85,111000008,디지털미디어시티역,58,127,111000006,수색역앞,58,127,112000002,84.0,서울75사1653,202108181745,1.0,202108181749,서울75사1653,2021081817,3
42769,202108181749,서울75사1653,0.507,126.894103,37.583047,111900090,1,4,100100073,86,111000006,수색역앞,58,127,111000004,수색교,58,127,111000008,85.0,서울75사1653,202108181746,1.0,202108181751,서울75사1653,2021081817,2


In [59]:
df_bus_100100073_weather = df_bus_100100073_complete.merge(df_weather_time_loc, left_on=['baseDateHour', 'nx_x', 'ny_x'], right_on=['baseDateHour', 'nx', 'ny'])
df_bus_100100073_weather = df_bus_100100073_weather.drop(columns=['preStation_x', 'preSeq', 'prePlainNo', 'preBaseDateTime', 'seqDiff'])

In [60]:
df_bus_100100073_weather.to_csv('./df_bus_100100073_weather.csv')

In [61]:
df_bus_100100073_weather['interval'].value_counts()

2     9608
1     8732
3     6158
4     2356
0     1178
5     1164
6      569
7      271
9      159
8      141
10     125
11      63
12      54
14      34
13      32
16      15
15      12
17       8
19       7
18       5
21       2
24       1
20       1
Name: interval, dtype: int64

In [80]:
df_bus_100100073_weather.columns

Index(['baseDateMinute', 'plainNo', 'fullSectDist', 'gpsX', 'gpsY',
       'sectionId', 'stopFlag', 'congetion', 'busRouteId', 'seq', 'station_x',
       'stationNm_x', 'nx_x', 'ny_x', 'station_y', 'stationNm_y', 'nx_y',
       'ny_y', 'nextBaseDateTime', 'nextPlainNo', 'baseDateHour', 'interval',
       'nx', 'ny', 'PTY', 'REH', 'RN1', 'T1H', 'UUU', 'VEC', 'VVV', 'WSD'],
      dtype='object')

In [79]:
df_bus_100100073_weather.sort_values(['plainNo', 'baseDateMinute']).head(10).drop(columns=['fullSectDist', 'stopFlag', 'busRouteId', 'station_x', 'nx_x', 'ny_x', 'station_y', 'nx_y', 'ny_y', 'nextBaseDateTime', 'nextPlainNo', 'baseDateHour', 'UUU', 'VVV', 'VEC'])

,baseDateMinute,plainNo,gpsX,gpsY,sectionId,congetion,seq,stationNm_x,stationNm_y,interval,nx,ny,PTY,REH,RN1,T1H,WSD
0,202108101042,서울70사6587,126.904464,37.575563,112602628,3,12,북가좌동삼거리,DMC래미안e.편한세상요진아파트,2,59,127,0,62,0,28.9,1.1
1,202108101044,서울70사6587,126.909702,37.571645,112602629,3,13,DMC래미안e.편한세상요진아파트,모래내시장.가좌역,2,59,127,0,62,0,28.9,1.1
2,202108101046,서울70사6587,126.914576,37.569268,112602300,3,14,모래내시장.가좌역,사천교,2,59,127,0,62,0,28.9,1.1
3,202108101048,서울70사6587,126.917966,37.567651,112603411,3,15,사천교,연희104고지앞.구성산회관,3,59,127,0,62,0,28.9,1.1
4,202108101051,서울70사6587,126.925254,37.56629,112602251,3,16,연희104고지앞.구성산회관,서대문우체국,2,59,127,0,62,0,28.9,1.1
5,202108101053,서울70사6587,126.931595,37.56224,112603415,3,17,서대문우체국,연세대앞,2,59,127,0,62,0,28.9,1.1
6,202108101055,서울70사6587,126.935377,37.559991,112603417,3,18,연세대앞,세브란스병원앞,1,59,127,0,62,0,28.9,1.1
7,202108101056,서울70사6587,126.940853,37.560835,112603422,3,19,세브란스병원앞,이대후문,1,59,127,0,62,0,28.9,1.1
8,202108101057,서울70사6587,126.942896,37.562349,112603553,3,20,이대후문,영천시장,6,59,127,0,62,0,28.9,1.1
118,202108101103,서울70사6587,126.961941,37.570299,112603555,3,21,영천시장,금화초등학교.서울시교육청,2,60,127,0,60,0,29.7,3.6


In [62]:
df_bus_100100073_weather[df_bus_100100073_weather['baseDateMinute'].str.contains('2021081617')]

,baseDateMinute,plainNo,fullSectDist,gpsX,gpsY,sectionId,stopFlag,congetion,busRouteId,seq,station_x,stationNm_x,nx_x,ny_x,station_y,stationNm_y,nx_y,ny_y,nextBaseDateTime,nextPlainNo,baseDateHour,interval,nx,ny,PTY,REH,RN1,T1H,UUU,VEC,VVV,WSD
11642,202108161703,서울70사6587,0.401,126.98748,37.567839,101603522,0,3,100100073,28,100000004,종로2가.삼일교,60,127,101000001,남대문세무서.서울백병원,60,127,202108161705,서울70사6587,2021081617,2,60,127,0,49,0,30.9,0.3,187,2.6,2.6
11643,202108161705,서울70사6587,1.004,126.987468,37.564891,101604636,1,3,100100073,29,101000001,남대문세무서.서울백병원,60,127,101000328,남산1호터널,60,127,202108161707,서울70사6587,2021081617,2,60,127,0,49,0,30.9,0.3,187,2.6,2.6
11644,202108161707,서울70사6587,2.585,126.992849,37.555661,102603021,0,3,100100073,30,101000328,남산1호터널,60,127,102000071,순천향대학병원.한남오거리,60,126,202108161710,서울70사6587,2021081617,3,60,127,0,49,0,30.9,0.3,187,2.6,2.6
11645,202108161739,서울70사6589,0.342,126.961941,37.570299,112603555,1,3,100100073,21,112000050,영천시장,60,127,112000051,금화초등학교.서울시교육청,60,127,202108161740,서울70사6589,2021081617,1,60,127,0,49,0,30.9,0.3,187,2.6,2.6
11646,202108161740,서울70사6589,0.604,126.964326,37.567901,101604870,1,3,100100073,22,112000051,금화초등학교.서울시교육청,60,127,101000262,서대문역사거리.농협중앙회,60,127,202108161742,서울70사6589,2021081617,2,60,127,0,49,0,30.9,0.3,187,2.6,2.6
11647,202108161742,서울70사6589,0.331,126.968493,37.567549,101604199,0,3,100100073,23,101000262,서대문역사거리.농협중앙회,60,127,100000380,서울역사박물관.경희궁앞,60,127,202108161743,서울70사6589,2021081617,1,60,127,0,49,0,30.9,0.3,187,2.6,2.6
11648,202108161743,서울70사6589,0.464,126.969995,37.56881,100602679,1,3,100100073,24,100000380,서울역사박물관.경희궁앞,60,127,100000384,광화문,60,127,202108161745,서울70사6589,2021081617,2,60,127,0,49,0,30.9,0.3,187,2.6,2.6
11649,202108161745,서울70사6589,0.525,126.976541,37.570067,100602877,1,3,100100073,25,100000384,광화문,60,127,100000386,종로1가,60,127,202108161748,서울70사6589,2021081617,3,60,127,0,49,0,30.9,0.3,187,2.6,2.6
11650,202108161748,서울70사6589,0.409,126.980911,37.570118,100602881,1,3,100100073,26,100000386,종로1가,60,127,100000388,종로2가,60,127,202108161749,서울70사6589,2021081617,1,60,127,0,49,0,30.9,0.3,187,2.6,2.6
11651,202108161749,서울70사6589,0.369,126.984668,37.570099,100601685,1,3,100100073,27,100000388,종로2가,60,127,100000004,종로2가.삼일교,60,127,202108161752,서울70사6589,2021081617,3,60,127,0,49,0,30.9,0.3,187,2.6,2.6


In [63]:
a = 1

In [64]:
df_bus_100100073_weather[df_bus_100100073_weather['sectionId']=='121607972']

,baseDateMinute,plainNo,fullSectDist,gpsX,gpsY,sectionId,stopFlag,congetion,busRouteId,seq,station_x,stationNm_x,nx_x,ny_x,station_y,stationNm_y,nx_y,ny_y,nextBaseDateTime,nextPlainNo,baseDateHour,interval,nx,ny,PTY,REH,RN1,T1H,UUU,VEC,VVV,WSD
282,202108101115,서울74사1363,0.667,127.026484,37.500633,121607972,1,3,100100073,59,121000012,지하철2호선강남역,61,125,121000014,논현역,61,125,202108101118,서울74사1363,2021081011,3,61,125,0,58,0,30.3,0.4,202,1,1.1
292,202108101139,서울74사2506,0.667,127.026335,37.50095,121607972,1,3,100100073,59,121000012,지하철2호선강남역,61,125,121000014,논현역,61,125,202108101142,서울74사2506,2021081011,3,61,125,0,58,0,30.3,0.4,202,1,1.1
313,202108101106,서울74사4110,0.667,127.026565,37.500461,121607972,1,3,100100073,59,121000012,지하철2호선강남역,61,125,121000014,논현역,61,125,202108101109,서울74사4110,2021081011,3,61,125,0,58,0,30.3,0.4,202,1,1.1
334,202108101125,서울74사5451,0.667,127.02637,37.500785,121607972,1,3,100100073,59,121000012,지하철2호선강남역,61,125,121000014,논현역,61,125,202108101128,서울74사5451,2021081011,3,61,125,0,58,0,30.3,0.4,202,1,1.1
378,202108101152,서울75사1375,0.667,127.026455,37.500695,121607972,1,3,100100073,59,121000012,지하철2호선강남역,61,125,121000014,논현역,61,125,202108101155,서울75사1375,2021081011,3,61,125,0,58,0,30.3,0.4,202,1,1.1
446,202108101245,서울70사6587,0.667,127.02637,37.500785,121607972,1,3,100100073,59,121000012,지하철2호선강남역,61,125,121000014,논현역,61,125,202108101248,서울70사6587,2021081012,3,61,125,0,57,0,30.7,0.4,202,1,1.1
477,202108101225,서울74사2510,0.667,127.026379,37.500856,121607972,1,3,100100073,59,121000012,지하철2호선강남역,61,125,121000014,논현역,61,125,202108101228,서울74사2510,2021081012,3,61,125,0,57,0,30.7,0.4,202,1,1.1
487,202108101239,서울74사2562,0.667,127.026511,37.500576,121607972,1,3,100100073,59,121000012,지하철2호선강남역,61,125,121000014,논현역,61,125,202108101242,서울74사2562,2021081012,3,61,125,0,57,0,30.7,0.4,202,1,1.1
495,202108101216,서울74사4605,0.667,127.026032,37.501594,121607972,0,3,100100073,59,121000012,지하철2호선강남역,61,125,121000014,논현역,61,125,202108101218,서울74사4605,2021081012,2,61,125,0,57,0,30.7,0.4,202,1,1.1
511,202108101200,서울74사7215,0.667,127.026386,37.50084,121607972,0,3,100100073,59,121000012,지하철2호선강남역,61,125,121000014,논현역,61,125,202108101202,서울74사7215,2021081012,2,61,125,0,57,0,30.7,0.4,202,1,1.1


In [65]:
from_gangnam = ['뱅뱅사거리', '래미안아파트.파이낸셜뉴스', '신분당선강남역', '지하철2호선강남역', '논현역', '신사역', '한남대교전망카페', '한남5거리', '순천향대학병원', '서울백병원.국가인권위.안중근활동터']
to_gangnam = ['남대문세무서.서울백병원', '남산1호터널', '순천향대학병원.한남오거리', '신사역.푸른저축은행', '논현역', '지하철2호선강남역', '신분당선강남역', '래미안아파트.파이낸셜뉴스', '뱅뱅사거리']

df_bus_100100073_weather[df_bus_100100073_weather['stationNm_x'].isin(from_gangnam+to_gangnam)]

,baseDateMinute,plainNo,fullSectDist,gpsX,gpsY,sectionId,stopFlag,congetion,busRouteId,seq,station_x,stationNm_x,nx_x,ny_x,station_y,stationNm_y,nx_y,ny_y,nextBaseDateTime,nextPlainNo,baseDateHour,interval,nx,ny,PTY,REH,RN1,T1H,UUU,VEC,VVV,WSD
126,202108101118,서울70사6587,1.004,126.98753,37.564985,101604636,1,4,100100073,29,101000001,남대문세무서.서울백병원,60,127,101000328,남산1호터널,60,127,202108101121,서울70사6587,2021081011,3,60,127,0,60,0,29.7,1.6,207,3.2,3.6
127,202108101121,서울70사6587,2.585,126.992849,37.555661,102603021,0,4,100100073,30,101000328,남산1호터널,60,127,102000071,순천향대학병원.한남오거리,60,126,202108101125,서울70사6587,2021081011,4,60,127,0,60,0,29.7,1.6,207,3.2,3.6
128,202108101133,서울74사1363,0.604,126.987851,37.564372,101600424,1,3,100100073,65,101000002,서울백병원.국가인권위.안중근활동터,60,127,100000001,종로2가사거리,60,127,202108101136,서울74사1363,2021081011,3,60,127,0,60,0,29.7,1.6,207,3.2,3.6
145,202108101151,서울74사1821,1.004,126.987465,37.564964,101604636,1,3,100100073,29,101000001,남대문세무서.서울백병원,60,127,101000328,남산1호터널,60,127,202108101153,서울74사1821,2021081011,2,60,127,0,60,0,29.7,1.6,207,3.2,3.6
146,202108101153,서울74사1821,2.585,126.992493,37.556479,102603021,0,3,100100073,30,101000328,남산1호터널,60,127,102000071,순천향대학병원.한남오거리,60,126,202108101157,서울74사1821,2021081011,4,60,127,0,60,0,29.7,1.6,207,3.2,3.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30569,202108110846,서울75사1364,1.37,127.020747,37.512647,121607076,1,3,100100073,61,121000016,신사역,61,126,122000408,한남대교전망카페,60,126,202108110851,서울75사1364,2021081108,5,61,126,0,78,0,25.9,-0.1,5,-2,2.1
30570,202108110858,서울75사1365,0.888,127.019981,37.514205,121600010,1,4,100100073,32,121000015,신사역.푸른저축은행,61,126,121000013,논현역,61,125,202108110902,서울75사1365,2021081108,4,61,126,0,78,0,25.9,-0.1,5,-2,2.1
30688,202108120856,서울75사1366,0.725,127.007987,37.533173,102602969,1,4,100100073,63,102000072,한남5거리,60,126,102000068,순천향대학병원,60,126,202108120859,서울75사1366,2021081208,3,60,126,0,68,0,27.5,-1.5,51,-1.2,2.1
30689,202108120859,서울75사1366,3.317,127.00573,37.536442,101604644,1,4,100100073,64,102000068,순천향대학병원,60,126,101000002,서울백병원.국가인권위.안중근활동터,60,127,202108120904,서울75사1366,2021081208,5,60,126,0,68,0,27.5,-1.5,51,-1.2,2.1
